<a href="https://colab.research.google.com/github/nguyenanhtienabcd/AIO2024_EXERCISE/blob/feature%2FMODULE7-WEEK3/m07w03_ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Object tracking - Simple version

In [1]:
from collections import defaultdict
import cv2
import numpy as np
import os
from ultralytics import YOLO


In [6]:
# load model YOLO 11 from YOLO
model = YOLO('yolo11l.pt')

# openr the video file
video_path = '/content/vtv24h.mp4'
cap = cv2.VideoCapture(video_path)


In [7]:
'''sử dung VideoWriter để lưu video kết quả đã được xử lý '''
# get video properties
width =  int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# create videowriter object - lưu lại video với số khung hình và kích thước khung hình là như nhau
video_name =  video_path.split('/')[-1] # lấy file của video "vtv24.mp4"
output_path = f'/content/{video_name.split(".")[0]}_output.mp4'
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

In [ ]:
# Store the track history
track_history = defaultdict(lambda: [])

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLO11 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True, show=False)

        # Get the boxes and track IDs (with error handling)
        boxes = results[0].boxes.xywh.cpu()
        try:
            track_ids = results[0].boxes.id
            if track_ids is not None:
                track_ids = track_ids.int().cpu().tolist()
            else:
                track_ids = []  # No tracks found in this frame
        except AttributeError:
            track_ids = []  # Handle case where tracking fails

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks only if we have valid tracking data
        if track_ids:
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point

                if len(track) > 120:  # retain 30 tracks for 30 frames
                    track.pop(0)

                # Draw the tracking lines
                points = np.hstack(track).astype(np.int32).reshape(-1, 1, 2)
                cv2.polylines(
                    annotated_frame,
                    [points],
                    isClosed=False,
                    color=(230, 230, 230),
                    thickness=4,
                )

        # Write the frame to output video
        out.write(annotated_frame)
    else:
        # Break the loop if the end of the video is reached
        break

# Release everything
cap.release()
out.release()
print(f"Video has been saved to {output_path}")


### một ví dụ về hàm default dictionary

In [5]:
from collections import defaultdict
import numpy as np

np.random.seed(42)

track_history = defaultdict(lambda: [])
print(track_history)

for i in range(1,7):
  track = track_history[i]
  print(track)

  # Generate random numbers using np.random.random() and then round
  track.append((np.random.randint(10), np.random.randint(10)))
  #print(track_history.values())

# chuyên về n hàng, 1 cột, 2 chiều
b = np.hstack(list(track_history.values()))
print(b)
c = b.astype(np.int32).reshape((-1, 1, 2))
print(c)

defaultdict(<function <lambda> at 0x7beda95ed120>, {})
[]
[]
[]
[]
[]
[]
[[6 3 7 4 6 9 2 6 7 4 3 7]]
[[[6 3]]

 [[7 4]]

 [[6 9]]

 [[2 6]]

 [[7 4]]

 [[3 7]]]
